# Rhizopus oryzae
<img src="RhozyMap.png" width="400" />

#### Help
1.) Molecular Formula of SMH

2.) Do we still fit β even if it has been calculated from anabolism?

3.) how do I incorporate the little amount of nitrogen available when fermentation egins for SMH, and how do do this so that biomass reflect the on the presence of this and so could plot its depletion behavioour. Because Rhozy grows in the presence of nitrogen

#### Assumptions
1.)

#### To do
1.) Molecular Formula of SMH

2.) Text background to the problem.

3.) Purpose (What are you modelling and what do you want to get out of the model?)

4.) References (Links)

In [1]:
import numpy as np
from numpy.linalg import solve
from scipy.optimize import fsolve, minimize
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import pandas
from ipywidgets import interact
from IPython.display import Image
# from __future__ import division
%matplotlib inline

In [2]:
c, h, o, n = 12, 1, 16, 14
MMs = c + 2*h + o
MMx = c + 1.8*h + 0.5*o + 0.2*n
MMe = c + 3*h + 0.5*o
MMn = n + 3*h
MMf = c + h + o
MMm = c + 1.5*h + 1.25*o
MMe,MMx,MMs,MMn,MMf, MMm

(23.0, 24.6, 30, 17, 29, 33.5)

### Predicting the Molecular formula of SMH

In [3]:
#Nitrgoen obtained by multiplyinh the protein by 16%
#g/l of nitrogen coverted into mol/L by deviding by MM on nitrogen(14)

In [4]:
datag = pandas.read_csv('growth.csv')
tg = datag['t'].T
Csugarg = datag['Cs']
Cnitrogeng = datag['Cn']

In [14]:
def growth (μ_max  = 0.02, θ_max = 0, Kms = 0.5, Kmn = 1,β =0.1):
    def response_fun(C):
        Cx, Cs, Cf, Ce, Cn = C
        if Cs <= 0:
            return [0,0,0,0,0]
        
        else:    
            μ = μ_max* (Cs/ (Kms + Cs))#*((1+ (Cf/Kmn))**(-1))
            θ = θ_max 
            Yxu = 0.2 #fix for SMH

            A1 = np.matrix([[-1, 1+α, 1, 0, 0, 0, 0],
                          [0, 0, -1, 1, 1, 1, 0],
                          [0, -γ, 1/3, -1, 0, 1/3, 2*P_O],
                          [0, β, 1/3, -1/4, -1/2, 2, -2],
                          [1, 0, 0, 0, 0, 0, 0],
                          [0, 1, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 1, 0, 0]])


            B1 = np.matrix([[0, 0, θ, 0, r_gluc, μ, r_ethanol]]).T

            r1 = np.linalg.solve(A1,B1)

            return[r1[1,0],
              -r1[0,0],
              r1[3,0],
              r1[4,0],
                  -Yxu*r1[1,0]]

    def ferm_eq(C,t):
        Cx, Cs, Cf, Ce, Cn = C[0],C[1], C[2], C[3], C[4]

        r=response_fun([Cx, Cs, Cf, Ce, Cn])     

        return [r[0]*Cx,
                r[1]*Cx,
                r[2]*Cx,
                r[3]*Cx,
                r[4]*Cx,]
    
    
    α, γ, P_O= 0.1, 2.5, 1.5#, μ_max, 0.1, θ, 0, 
    r_gluc = ((10-0.11)/((0.45/(0.05))*24))*(MMx/MMs) #cx,massx,50ml,time#
    r_ethanol =((0.2-0)/((0.45/(0.05))*24))*(MMx/MMe) #NB: recheck: dvide it by 10 to match the amount of s i.e ferm 100g_s an 10 growth (look into this)
    Cso,Cxo,Cno,Cfo,Ceo = 10/MMs, 0.45/(0.05*MMx), (0.16*0.44)/14, 0, 0 #Fix initial Cx value. is it zero or?
                                                                      #Cn devided by 14 for nitrogen 
    Co=[Cxo, Cso, Cfo,Ceo, Cno]                     # [Cxo, Cso, Cfo, Ceo,Cno] in cmol/L
    tspan=np.linspace(0,25 ,100)           # define the timespan of the integration
    Ci = odeint(ferm_eq, Co, tspan)             # integration via odeint that calls apon ferm_eq

    Cx, Cs, Cf, Ce, Cn = Ci.T

    plt.figure(1)
    
    fig, ax1 = plt.subplots()
    
    color = 'tab:red'
    ax1.set_xlabel('time (h)')
    ax1.set_ylabel('glucose (cmol/L)', color=color)
    ax1.plot(tspan, Cs, color=color, label='Cs')
    ax1.plot(tg, Csugarg, '*', color=color)
    ax1.tick_params(axis='y', labelcolor=color)

#     ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

#     color = 'tab:green'
#     ax2.set_ylabel('Nitrogen', color=color)  # we already handled the x-label with ax1
#     ax2.plot(tspan, Cn, color=color, label='Cn') #fix the nitrogen data, figure it out
#     ax2.plot(tg, Cnitrogeng, '*', color=color)
#     ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped

    
interact(growth, μ_max  = (0,2,0.05), θ_max = (0,10,0.05), Kms = (0, 3.3, 0.01),Kmn =  (0,1.3,0.01),β = (0,3,0.01))
plt.show()

interactive(children=(FloatSlider(value=0.02, description='μ_max', max=2.0, step=0.05), FloatSlider(value=0.0,…

In [13]:
# Image("growth.png")

 <tr>
    <td> <img src="growth.png" alt="Drawing" style="width: 400px;"/> </td>
    </tr>

# SMH 20% v/v (growth phase)

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

# SMH 20% v/v (production phase)

In [7]:
data = pandas.read_csv('SMH.csv')
t = data['t'].T
Csugar = data['Cs']
Cfumarate = data['Cf']
Cethanol = data['Ce']
Cmalic = data['Cm']

In [8]:
def SMH (μ_max  = 0.02, θ_max = 0, Kms = 0.5, Kmn = 1,β =0.1):
    def response_fun(C):
        Cx, Cs, Cf, Ce, Cn = C
        if Cs <= 0:
            return [0,0,0,0,0]
        
        else:    
            μ = μ_max* (Cs/ (Kms + Cs))#*((1+ (Cf/Kmn))**(-1))
            θ = θ_max
            Yxu = 0.2 #fix for SMH

            A1 = np.matrix([[-1, 1+α, 1, 0, 0, 0, 0],
                          [0, 0, -1, 1, 1, 1, 0],
                          [0, -γ, 1/3, -1, 0, 1/3, 2*P_O],
                          [0, β, 1/3, -1/4, -1/2, 2, -2],
                          [1, 0, 0, 0, 0, 0, 0],
                          [0, 1, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 1, 0, 0]])


            B1 = np.matrix([[0, 0, θ, 0, r_gluc, μ, r_ethanol]]).T

            r1 = np.linalg.solve(A1,B1)

            return[r1[1,0],
              -r1[0,0],
              r1[3,0],
              r1[4,0],
                  -Yxu*r1[1,0]]

    def ferm_eq(C,t):
        Cx, Cs, Cf, Ce, Cn = C[0],C[1], C[2], C[3], C[4]

        r=response_fun([Cx, Cs, Cf, Ce, Cn])     

        return [r[0]*Cx,
                r[1]*Cx,
                r[2]*Cx,
                r[3]*Cx,
                r[4]*Cx,]
    
    
    α, γ, P_O= 0.1, 2.5, 1.5#, μ_max, 0.1, θ, 0, 
    r_gluc = ((100-20)/((0.45/(0.04))*145))*(MMx/MMs) #cx,massx,40ml,time#
    r_ethanol =((2-0)/((0.45/(0.04))*145))*(MMx/MMe)
    Cso,Cxo,Cno,Cfo,Ceo = 97/MMs, 0.45/(0.04*MMx), (0.16*0.15)/14, 0, 0 #Cn devided by 14 for nitrogen 

    Co=[Cxo, Cso, Cfo,Ceo, Cno]                     # [Cxo, Cso, Cfo, Ceo,Cno] in cmol/L
    tspan=np.linspace(0,150 ,100)           # define the timespan of the integration
    Ci = odeint(ferm_eq, Co, tspan)             # integration via odeint that calls apon ferm_eq

    Cx, Cs, Cf, Ce, Cn = Ci.T

    plt.figure(1)

    plt.plot(tspan, Cs, color='red', label='Cs')
    plt.plot(t, Csugar, '*', color='red')
    
    plt.plot(tspan, Cf, color='green', label='Cf')
    plt.plot(t, Cfumarate, '*', color='green')
    
    plt.plot(tspan, Ce, color='black', label='Ce')
    plt.plot(t, Cethanol, '*', color='black')
    
#     plt.plot(tspan, Cm, color='black', label='Cm')
#     plt.plot(t, Cmalic, '*', color='blue')

    plt.legend(loc='best')
    plt.ylabel('Concentration cmol/L') 
    plt.xlabel('time (h)') 

    
interact(SMH, μ_max  = (0,2,0.05), θ_max = (0,10,0.05), Kms = (0, 3.3, 0.01),Kmn =  (0,1.3,0.01),β = (0,3,0.01))
plt.show()

interactive(children=(FloatSlider(value=0.02, description='μ_max', max=2.0, step=0.05), FloatSlider(value=0.0,…

 <tr>
    <td> <img src="smh20.png" alt="Drawing" style="width: 400px;"/> </td>
    </tr>

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

# Yeast (production phase)

In [9]:
data1 = pandas.read_csv('yeast.csv')
t1 = data1['t'].T
Csugar1 = data1['Cs']
Cfumarate1 = data1['Cf']
Cethanol1 = data1['Ce']
Cmalic1 = data1['Cm']

In [10]:
def yeast (μ_max  = 0.02, θ_max = 0, Kms = 0.5, Kmn = 1,β =0.1):
    def response_fun(C):
        Cx, Cs, Cf, Ce, Cn = C
        if Cs <= 0:
            return [0,0,0,0,0]
        
        else:    
            μ = μ_max* (Cs/ (Kms + Cs))#*((1+ (Cf/Kmn))**(-1))
            θ = θ_max
            Yxu = 0.2 #fix for yeast and recalculate β

            A1 = np.matrix([[-1, 1+α, 1, 0, 0, 0, 0],
                          [0, 0, -1, 1, 1, 1, 0],
                          [0, -γ, 1/3, -1, 0, 1/3, 2*P_O],
                          [0, β, 1/3, -1/4, -1/2, 2, -2],
                          [1, 0, 0, 0, 0, 0, 0],
                          [0, 1, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 1, 0, 0]])


            B1 = np.matrix([[0, 0, θ, 0, r_gluc, μ, r_ethanol]]).T

            r1 = np.linalg.solve(A1,B1)

            return[r1[1,0],
              -r1[0,0],
              r1[3,0],
              r1[4,0],
                  -Yxu*r1[1,0]]

    def ferm_eq(C,t):
        Cx, Cs, Cf, Ce, Cn = C[0],C[1], C[2], C[3], C[4]

        r=response_fun([Cx, Cs, Cf, Ce, Cn])     

        return [r[0]*Cx,
                r[1]*Cx,
                r[2]*Cx,
                r[3]*Cx,
                r[4]*Cx,]
    
    
    α, γ, P_O= 0.1, 2.5, 1.5#, μ_max, 0.1, θ, 0, 
    r_gluc = ((100-10)/((0.45/(0.04))*88))*(MMx/MMs) #cx,massx,50ml,time#
    r_ethanol =((28-0)/((0.45/(0.04))*88))*(MMx/MMe) #NB: chane to malic spec cause it is constant. remember to also change matrix
    Cso,Cxo,Cno,Cfo,Ceo = 100/MMs, 0.45/(0.04*MMx), 0, 0, 0

    Co=[Cxo, Cso, Cfo,Ceo, Cno]                     # [Cxo, Cso, Cfo, Ceo,Cno] in cmol/L
    tspan=np.linspace(0,100 ,100)           # define the timespan of the integration
    Ci = odeint(ferm_eq, Co, tspan)             # integration via odeint that calls apon ferm_eq

    Cx, Cs, Cf, Ce, Cn = Ci.T

    plt.figure(2)

    plt.plot(tspan, Cs, color='red', label='Cs')
    plt.plot(t1, Csugar1, '*', color='red')
    
    plt.plot(tspan, Cf, color='green', label='Cf')
    plt.plot(t1, Cfumarate1, '*', color='green')
    
    plt.plot(tspan, Ce, color='black', label='Ce')
    plt.plot(t1, Cethanol1, '*', color='black')
    
#     plt.plot(tspan, Cm, color='black', label='Cm')
#     plt.plot(t, Cmalic1, '*', color='blue')

    plt.legend(loc='best')
    plt.ylabel('Concentration cmol/L') 
    plt.xlabel('time (h)') 

    
interact(yeast, μ_max  = (0,2,0.05), θ_max = (0,10,0.05), Kms = (0, 3.3, 0.01),Kmn =  (0,1.3,0.01),β = (0,3,0.01))
plt.show()

interactive(children=(FloatSlider(value=0.02, description='μ_max', max=2.0, step=0.05), FloatSlider(value=0.0,…

 <tr>
    <td> <img src="yeast.png" alt="Drawing" style="width: 400px;"/> </td>
    </tr>

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

# Urea (production phase)

In [11]:
data2 = pandas.read_csv('urea.csv')
t2 = data2['t'].T
Csugar2 = data2['Cs']
Cfumarate2 = data2['Cf']
Cethanol2 = data2['Ce']
Cmalic2 = data2['Cm']

In [12]:
def Urea (μ_max  = 0.02, θ_max = 0, Kms = 0.5, Kmn = 1,β =0.1):
    def response_fun(C):
        Cx, Cs, Cf, Ce, Cn = C
        if Cs <= 0:
            return [0,0,0,0,0]
        
        else:    
            μ = μ_max* (Cs/ (Kms + Cs))#*((1+ (Cf/Kmn))**(-1))
            θ = θ_max
            Yxu = 0.2 #fix for yeast and recalculate β

            A1 = np.matrix([[-1, 1+α, 1, 0, 0, 0, 0],
                          [0, 0, -1, 1, 1, 1, 0],
                          [0, -γ, 1/3, -1, 0, 1/3, 2*P_O],
                          [0, β, 1/3, -1/4, -1/2, 2, -2],
                          [1, 0, 0, 0, 0, 0, 0],
                          [0, 1, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 1, 0, 0]])


            B1 = np.matrix([[0, 0, θ, 0, r_gluc, μ, r_ethanol]]).T

            r1 = np.linalg.solve(A1,B1)

            return[r1[1,0],
              -r1[0,0],
              r1[3,0],
              r1[4,0],
                  -Yxu*r1[1,0]]

    def ferm_eq(C,t):
        Cx, Cs, Cf, Ce, Cn = C[0],C[1], C[2], C[3], C[4]

        r=response_fun([Cx, Cs, Cf, Ce, Cn])     

        return [r[0]*Cx,
                r[1]*Cx,
                r[2]*Cx,
                r[3]*Cx,
                r[4]*Cx,]
    
    
    α, γ, P_O= 0.1, 2.5, 1.5#, μ_max, 0.1, θ, 0, 
    r_gluc = ((100-10)/((0.45/(0.04))*88))*(MMx/MMs) #cx,massx,50ml,time#
    r_ethanol =((20-0)/((0.45/(0.04))*88))*(MMx/MMe) #NB: chane to malic spec cause it is constant. remember to also change matrix
    Cso,Cxo,Cno,Cfo,Ceo = 100/MMs, 0.45/(0.04*MMx), 0, 0, 0

    Co=[Cxo, Cso, Cfo,Ceo, Cno]                     # [Cxo, Cso, Cfo, Ceo,Cno] in cmol/L
    tspan=np.linspace(0,100 ,100)           # define the timespan of the integration
    Ci = odeint(ferm_eq, Co, tspan)             # integration via odeint that calls apon ferm_eq

    Cx, Cs, Cf, Ce, Cn = Ci.T

    plt.figure(3)

    plt.plot(tspan, Cs, color='red', label='Cs')
    plt.plot(t2, Csugar2, '*', color='red')
    
    plt.plot(tspan, Cf, color='green', label='Cf')
    plt.plot(t2, Cfumarate2, '*', color='green')
    
    plt.plot(tspan, Ce, color='black', label='Ce')
    plt.plot(t2, Cethanol2, '*', color='black')
    
#     plt.plot(tspan, Cm, color='black', label='Cm')
#     plt.plot(t, Cmalic2, '*', color='blue')

    plt.legend(loc='best')
    plt.ylabel('Concentration cmol/L') 
    plt.xlabel('time (h)') 

    
interact(Urea, μ_max  = (0,2,0.05), θ_max = (0,10,0.05), Kms = (0, 3.3, 0.01),Kmn =  (0,1.3,0.01),β = (0,3,0.01))
plt.show()

interactive(children=(FloatSlider(value=0.02, description='μ_max', max=2.0, step=0.05), FloatSlider(value=0.0,…

 <tr>
    <td> <img src="urea.png" alt="Drawing" style="width: 400px;"/> </td>
    </tr>

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 